In [2]:
import matplotlib.pyplot as plt
import os.path as osp
import numpy as np
num_gpus = 4
classes = ("Meat", "Nuts/seeds", "Eggs", "Beans/lentils/peas", "Fruit", "Grain", "Vegetables", "Dairy", "Sauce/Spread", "Soup/Drink")

In [3]:
from mmcv import Config
cfg = Config.fromfile('mmclassification/configs/resnet/resnet101_4xb16_foodseg103.py')
name = "mobile_v3_l_4xb16_combined"
import os
work_dir = './work_dirs/' + name
checkpoint_file = os.path.join(work_dir, "latest.pth")
if(not osp.exists(checkpoint_file)):
    checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/mobilenet_v3/convert/mobilenet_v3_large-3ea3c186.pth'
    print("Loading pretrained weights")

In [4]:
cfg.model.head.num_classes = len(classes)

# Load the pre-trained model's checkpoint.
cfg.model.backbone = dict(type='MobileNetV3', arch='large')
cfg.model.backbone['init_cfg'] = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.model.head.in_channels=960

# Specify sample size and number of workers.
cfg.data.samples_per_gpu = 16
cfg.checkpoint_config = dict(interval=10)
cfg.log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
cfg.data.workers_per_gpu = 4
cfg.runner = dict(type='EpochBasedRunner', max_epochs=40)
print(f'Config:\n{cfg.pretty_text}')
config_path = "mmclassification/configs/food103configs/" + name + ".py"
with open(config_path, "w") as f:
    f.write(cfg.pretty_text)

Config:
dataset_type = 'FoodSeg103'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, -1)),
    dict(type='CenterCrop', crop_size=224),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    samples_per_gpu=16,
    workers_per_gp

In [ ]:
f"mmclassification/tools/dist_train.sh {config_path} {num_gpus}"

In [ ]:
!mmclassification/tools/dist_train.sh {config_path} {num_gpus}

In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
checkpoint_file = os.path.join(work_dir, "latest.pth")


In [7]:
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint

from mmcls.datasets.pipelines import Compose

def inference_model(model, img):
    """Inference image(s) with the classifier.

    Args:
        model (nn.Module): The loaded classifier.
        img (str/ndarray): The image filename or loaded image.
    """
    cfg = model.cfg
    device = next(model.parameters()).device  # model device
    # build the data pipeline
    if isinstance(img, str):
        if cfg.data.test.pipeline[0]['type'] != 'LoadImageFromFile':
            cfg.data.test.pipeline.insert(0, dict(type='LoadImageFromFile'))
        data = dict(img_info=dict(filename=img), img_prefix=None)
    else:
        if cfg.data.test.pipeline[0]['type'] == 'LoadImageFromFile':
            cfg.data.test.pipeline.pop(0)
        data = dict(img=img)
    test_pipeline = Compose(cfg.data.test.pipeline)
    data = test_pipeline(data)
    data = collate([data], samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]

    # forward the model
    with torch.no_grad():
        scores = model(return_loss=False, **data)[0]
    return scores

In [9]:
import pandas as pd
from mmcls.apis import init_model
import torch as torch
import glob
import mmcv
labels_db = pd.read_csv("food103labels.csv",sep=',')
model = init_model(config_path, checkpoint_file, device='cuda:0')
LABELS = list(model.CLASSES)
np_acc = np.zeros(len(LABELS))
count = 0
file_list = glob.glob("SegCombined/Images/img_dir/test/*")
all_predictions = np.zeros(shape=(len(file_list), len(LABELS)))
all_gt = np.zeros(shape=(len(file_list), len(LABELS)))
def get_label(file_path):    
    file_name = file_path.split('/')[-1]
    numpy_array = labels_db.loc[labels_db['filename'] == file_name][LABELS].to_numpy().astype('int64')
    return numpy_array
for file in file_list:
    img_array =  mmcv.imread(file)
    preds = inference_model(model, file)
    all_predictions[count, :] = preds
    gt_label = get_label(file).squeeze()
    all_gt[count, :] = gt_label
    count+=1
    print(count, end='\r')



/nobackup/hli5/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/cnn/bricks/hsigmoid.py:31: UserWarning: In MMCV v1.4.4, we modified the default value of args to align with PyTorch official. Previous Implementation: Hsigmoid(x) = min(max((x + 1) / 2, 0), 1). Current Implementation: Hsigmoid(x) = min(max((x + 3) / 6, 0), 1).
  warnings.warn(


load checkpoint from local path: ./work_dirs/mobile_v3_l_4xb16_combined/latest.pth


In [11]:
from mmseg.core.evaluation import mean_ap

mean_average_precision = mean_ap.mAP(all_predictions, all_gt)
print(mean_average_precision)
print(mean_average_precision.mean())

[0.92547863 0.49709374 0.38146654 0.08173596 0.86636094 0.84972295
 0.98252312 0.63917717 0.53379831 0.73247498]
0.6489832341851518


In [13]:
all_predictions[all_predictions>=0.5] = 1
all_predictions[all_predictions<0.5] = 0

acc = (all_predictions==all_gt).sum(axis=0)
acc = acc/len(file_list)
for i in range(len(LABELS)):
    print(f"Accuracy of {LABELS[i]}: {acc[i]}")
print(acc.mean())

[0.84967646 0.95968143 0.95520159 0.97859632 0.8765555  0.77003484
 0.89696366 0.86311598 0.87356894 0.93081135]
0.8954206072672971
